# Титул: Домашнее задание №9
## Исполнитель: Ходжазода Садриддин

#### Домашняя работа по plotly
На основе данных из таблицы track создайте столбчатую диаграмму, показывающую общую сумму минут (длительность треков) в разбивке по жанрам.
На основе данных из таблицы track создайте диаграмму рассеивания, показывающую соотношение минут и мегабайтов.
На основе данных из таблицы invoice создайте коробчатую диаграмму, показывающую распределение дохода (поле total) в разбивке по странам.
На основе данных из таблицы invoice создайте линейчатую диаграмму, показывающую изменение дохода по месяцам за 2025 год.
На основе данных из таблицы invoice, создайте круговую диаграмму, показывающую соотношение продаж в США и в остальном мире. Т.е. ваша круговая диаграмма должна состоять лишь из двух секций: США и остальной мир.
Файл решения должен быть в формате блокнота 'ipynb' (jupyter notebook). В блокноте должно быть следующее:
* титул - "Домашнее задание №"
* исполнитель - "исполнитель: Имя и Фамилия"
* номер и полное описание задачи
* код решения

Создайте в вашем репозитории новую ветку и сохраните файл с кодом решения перечисленных задач в эту ветку. Сделайте коммит, пуш и создайте pull request. В классруме прикрепите скриншот вкладки files changed вашего pull request-а и ссылку на репозиторий.

In [2]:
import sqlalchemy as db
import pandas as pd
import plotly.express as px

def read_credentials():
    with open('credentials.txt') as f:
        connection_string = f.read().strip()
    return connection_string

def connect_to(db_name):
    conn = None
    try:
        if db_name == 'postgres':
            connection_string = read_credentials()
            engine = db.create_engine(connection_string)
            conn = engine.connect()
            print("Successfully connected to the database")
        else:
            print(f"Database {db_name} is not supported.")
    except Exception as e:
        print(f"An error occurred while connecting to the database: {e}")
    
    return conn

## Задание 1.

In [12]:
connection_string = read_credentials()
engine = db.create_engine(connection_string)
conn = engine.connect()

query = """
select 
    genre.name as genre
    , sum(track.milliseconds) / 60000 as totalminutes
from track
join genre on track.genre_id = genre.genre_id
group by genre.name
order by totalminutes desc;
"""

df = pd.read_sql_query(query, conn)

conn.close()

In [11]:
import plotly.express as px

fig = px.bar(df, x='genre', y='totalminutes', 
             title='Общая длительность треков по жанрам (в минутах)',
             labels={'genre': 'Жанр', 'totalminutes': 'Суммарная длительность (минуты)'},
             color='totalminutes')

fig.show()

## Задание 2.

In [3]:
connection_string = read_credentials()
engine = db.create_engine(connection_string)
conn = engine.connect()

query = """
select 
    round(t.milliseconds/60000., 2) as minutes
    , round(t.bytes/1024/1024., 2) as megabytes
from track t;
"""

df2 = pd.read_sql_query(query, conn)
conn.close()

In [5]:
fig = px.scatter(df2, x = 'minutes', y = 'megabytes',
                 title='Соотношение длительности и размера',
                 labels={'minutes': 'Длительность в минутах', 'megabytes': 'Размер в мегабайтах'})

fig.show()

## Задание 3.

In [7]:
connection_string = read_credentials()
engine = db.create_engine(connection_string)
conn = engine.connect()

query = """
select
    i.total as total
    , i.billing_country as country
from invoice i;
"""

df3 = pd.read_sql_query(query, conn)
conn.close()

In [8]:
fig = px.box(df3, x='country', y='total',
             title='Распеределине дохода по странам',
             labels={'country': 'Страны', 'total': 'Доход'})

fig.show()

## Задание 4.

In [12]:
connection_string = read_credentials()
engine = db.create_engine(connection_string)
conn = engine.connect()

query = """
select 
    extract(month from invoice_date) as month
    , sum(total) as total_income
from invoice
where extract(year from invoice_date) = 2025
group by extract(month from invoice_date)
order by month;
"""

df4 = pd.read_sql_query(query, conn)
conn.close()

In [13]:
fig = px.line(df4, x='month', y='total_income',
              title='Изменение дохода по месяцам за 2025 год',
              labels={'month': 'Месяц', 'total_income': 'Доход'})

fig.show()

## Задание 5.

In [17]:
connection_string = read_credentials()
engine = db.create_engine(connection_string)
conn = engine.connect()

query = """
select 
    case 
        when billing_country = 'USA' then 'США'
        else 'Остальной мир'
    end as region,
    sum(total) as total_sales
from invoice
group by region;
"""

df5 = pd.read_sql_query(query, conn)

conn.close()

In [18]:
fig = px.pie(df5, values='total_sales', names='region',
             title='Соотношение продаж в США и в остальном мире')

fig.show()